In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dataset.csv")
# Display the first few rows of the DataFrame
print(df.head())

           frame_path  left_state  left_hold_duration  right_state   
0  frames/frame_0.jpg           0                 0.0            0  \
1  frames/frame_1.jpg           0                 0.0            0   
2  frames/frame_2.jpg           0                 0.0            0   
3  frames/frame_3.jpg           0                 0.0            0   
4  frames/frame_4.jpg           0                 0.0            0   

   right_hold_duration  
0                  0.0  
1                  0.0  
2                  0.0  
3                  0.0  
4                  0.0  


In [3]:
# Load the image data

from PIL import Image
import numpy as np
df['frame_path'] = df['frame_path'].apply(lambda x: np.array(Image.open(x)))

In [4]:
# normalize the image data
df['frame_path'] = df['frame_path'].apply(lambda x: x/255)

In [6]:
# Columns to be dropped
drop_columns = [
                'left_hold_duration',
                'right_hold_duration']

# Drop the columns
df = df.drop(columns=drop_columns)

print(df.head())


                                          frame_path  left_state  right_state
0  [[[0.9215686274509803, 0.9176470588235294, 0.9...           0            0
1  [[[0.2, 0.5764705882352941, 0.7647058823529411...           0            0
2  [[[0.9686274509803922, 0.9450980392156862, 0.9...           0            0
3  [[[0.0392156862745098, 0.03529411764705882, 0....           0            0
4  [[[0.9294117647058824, 0.9254901960784314, 0.9...           0            0


In [7]:
# Create a new column 'empty_state' initialized with zeros
df['empty_state'] = 0

# Iterate through rows
for i in range(len(df)):
    # If 'left' and 'right' are the same
    if df.loc[i, 'left_state'] == df.loc[i, 'right_state']:
        # Replace 'left' and 'right' with 0
        df.loc[i, 'left_state'] = 0
        df.loc[i, 'left_state'] = 0
        # Assign 'empty_state' to 1
        df.loc[i, 'empty_state'] = 1
    
    # else assign 'empty_state' to 0
    else:
        df.loc[i, 'empty_state'] = 0

# Display the first few rows of the DataFrame
print(df.head())

                                          frame_path  left_state  right_state   
0  [[[0.9215686274509803, 0.9176470588235294, 0.9...           0            0  \
1  [[[0.2, 0.5764705882352941, 0.7647058823529411...           0            0   
2  [[[0.9686274509803922, 0.9450980392156862, 0.9...           0            0   
3  [[[0.0392156862745098, 0.03529411764705882, 0....           0            0   
4  [[[0.9294117647058824, 0.9254901960784314, 0.9...           0            0   

   empty_state  
0            1  
1            1  
2            1  
3            1  
4            1  


In [8]:
df.describe()

,left_state,right_state,empty_state
count,10263.000000,10263.000000,10263.000000
mean,0.110104,0.247978,0.641918
std,0.313035,0.431860,0.479460
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000
75%,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000


In [9]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
# Define features (X) and labels (Y) for training set
X_train = train_data['frame_path']
Y_train = train_data.drop('frame_path', axis=1)

# Define features (X) and labels (Y) for testing set
X_test = test_data['frame_path']
Y_test = test_data.drop('frame_path', axis=1)

In [11]:
X_train = np.stack(train_data['frame_path'])
X_test = np.stack(test_data['frame_path'])

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1:])),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),    # Flattening the 3D outputs of the previous layer
    
    Dense(512, activation='relu'),
    Dropout(0.5),

    Dense (256, activation='relu'),
    Dropout(0.5),
    
    Dense (256, activation='relu'),
    Dropout(0.5),

    Dense(Y_train.shape[1], activation='sigmoid')   # number of output neurons equals to number of key states
])



In [13]:

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy',   # use binary_crossentropy for multi-label classification
              metrics=['accuracy'])

# Print a summary of the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 238, 318, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 119, 159, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 119, 159, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 117, 157, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 58, 78, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 58, 78, 64)        0

In [14]:
# Train the model
history = model.fit(X_train, Y_train, validation_split=0.2, epochs=20, batch_size=16)

Epoch 1/20
411/411 [==============================] - 31s 58ms/step - loss: 0.5131 - accuracy: 0.6355 - val_loss: 0.4460 - val_accuracy: 0.6328
Epoch 2/20
411/411 [==============================] - 13s 31ms/step - loss: 0.4247 - accuracy: 0.6736 - val_loss: 0.3819 - val_accuracy: 0.6998
Epoch 3/20
411/411 [==============================] - 13s 31ms/step - loss: 0.3927 - accuracy: 0.7031 - val_loss: 0.3623 - val_accuracy: 0.7168
Epoch 4/20
411/411 [==============================] - 13s 31ms/step - loss: 0.3731 - accuracy: 0.7214 - val_loss: 0.3468 - val_accuracy: 0.7400
Epoch 5/20
411/411 [==============================] - 13s 31ms/step - loss: 0.3526 - accuracy: 0.7430 - val_loss: 0.3605 - val_accuracy: 0.7199
Epoch 6/20
411/411 [==============================] - 13s 31ms/step - loss: 0.3394 - accuracy: 0.7585 - val_loss: 0.3383 - val_accuracy: 0.7515
Epoch 7/20
411/411 [==============================] - 13s 31ms/step - loss: 0.3199 - accuracy: 0.7696 - val_loss: 0.3274 - val_accuracy:

In [15]:
model.save('model3.h5')

In [16]:
# evaluate the model
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])

Test loss: 0.33476969599723816


In [17]:
# calculate accuracy
print('Test accuracy:', score[1])

Test accuracy: 0.7939600348472595
